## ROI on Aliyun Data

In [47]:
from pathlib import Path
import dask.dataframe as dd

In [48]:
# get the data from a hard dr
folder_path = Path("/Volumes/Backup/aliyun_data")

In [52]:
# Load data using Dask
raw_sample = dd.read_csv(f'{folder_path}/raw_sample.csv')
ad_feature = dd.read_csv(f'{folder_path}/ad_feature.csv')
behavior_log = dd.read_csv(f'{folder_path}/behavior_log.csv')

In [35]:
# Filter clicks in raw_sample
raw_sample_clicks = raw_sample[raw_sample['clk'] == 1]
# Filter purchases in the behavior log
behavior_log_purchases = behavior_log[behavior_log['btag'] == 'buy']

In [41]:
# the raw data and behavior log of the users who click the ads and made the purchase
raw_log_with_clicks_purchase = raw_sample_clicks.merge(behavior_log_purchases, on='user')

In [43]:
# Merge with ad_feature to get the price of the items
clicks_and_purchases_with_price = raw_log_with_clicks_purchase.merge(ad_feature, on='adgroup_id')

In [44]:
# Calculate total revenue
revenue = clicks_and_purchases_with_price['price'].sum().compute()
print(f"Total Revenue: {revenue}")

Total Revenue: 11724919517.489998


In [ ]:
# # Calculate total impressions from raw_sample
# total_impressions = raw_sample.shape[0].compute()

# # Assuming a given cost per impression
# cost_per_impression = 100  # Example value
# cost = total_impressions * cost_per_impression
# print(f"Total Cost: {cost}")

# # Compute ROI
# roi = (revenue - cost) / cost * 100
# print(f"ROI: {roi}%")

## Analysis on Kaggle Data
Data: https://www.kaggle.com/datasets/loveall/clicks-conversion-tracking/data

In [12]:
import pandas as pd

# Load the dataset
df = pd.read_csv('KAG_conversion_data.csv')
print(df.head())

    ad_id  xyz_campaign_id  fb_campaign_id    age gender  interest  \
0  708746              916          103916  30-34      M        15   
1  708749              916          103917  30-34      M        16   
2  708771              916          103920  30-34      M        20   
3  708815              916          103928  30-34      M        28   
4  708818              916          103928  30-34      M        28   

   Impressions  Clicks  Spent  Total_Conversion  Approved_Conversion  
0         7350       1   1.43                 2                    1  
1        17861       2   1.82                 2                    0  
2          693       0   0.00                 1                    0  
3         4259       1   1.25                 1                    0  
4         4133       1   1.29                 1                    1  


In [3]:
# Group the data by campaign ID
campaign_group = df.groupby('xyz_campaign_id').agg({
    'ad_id': 'count',
    'Impressions': 'sum',
    'Clicks': 'sum',
    'Total_Conversion': 'sum',
    'Approved_Conversion': 'sum',
    'Spent': 'sum'
}).reset_index()

In [4]:
campaign_group

,xyz_campaign_id,ad_id,Impressions,Clicks,Total_Conversion,Approved_Conversion,Spent
0,916,54,482925,113,58,24,149.710001
1,936,464,8128187,1984,537,183,2893.369999
2,1178,625,204823716,36068,2669,872,55662.149959


### The User Funnel
- On the Overall Data
- Campaign-Level

In [5]:
# Calculate the funnel metrics for the overall data
total_impressions = df['Impressions'].sum()
total_clicks = df['Clicks'].sum()
total_conversions = df['Total_Conversion'].sum()
total_approved_conversions = df['Approved_Conversion'].sum()

ctr = (total_clicks / total_impressions) * 100 if total_impressions > 0 else 0  # Click-Through Rate (CTR)
conversion_rate = (total_conversions / total_clicks) * 100 if total_clicks > 0 else 0  # Conversion Rate from clicks to conversions
approval_rate = (total_approved_conversions / total_conversions) * 100 if total_conversions > 0 else 0  # Approval Rate from conversions to approved conversions

print(f'Total Impressions: {total_impressions}')
print(f'Total Clicks: {total_clicks}')
print(f'Total Conversions: {total_conversions}')
print(f'Total Approved Conversions: {total_approved_conversions}')
print(f'Click-Through Rate (CTR): {ctr:.2f}%')
print(f'Conversion Rate: {conversion_rate:.2f}%')
print(f'Approval Rate: {approval_rate:.2f}%')

Total Impressions: 213434828
Total Clicks: 38165
Total Conversions: 3264
Total Approved Conversions: 1079
Click-Through Rate (CTR): 0.02%
Conversion Rate: 8.55%
Approval Rate: 33.06%


In [13]:
# Group by campaign ID to calculate funnel metrics for each campaign
funnel = df.groupby('xyz_campaign_id').agg(
    Impressions=('Impressions', 'sum'),
    Clicks=('Clicks', 'sum'),
    Total_Conversions=('Total_Conversion', 'sum'),
    Approved_Conversions=('Approved_Conversion', 'sum')
).reset_index()

funnel['Click-Through Rate (CTR)'] = funnel['Clicks'] / funnel['Impressions']
funnel['Conversion Rate (CR)'] = funnel['Total_Conversions'] / funnel['Clicks']
funnel['Approval Rate (AR)'] = funnel['Approved_Conversions'] / funnel['Total_Conversions']

print(funnel)

   xyz_campaign_id  Impressions  Clicks  Total_Conversions  \
0              916       482925     113                 58   
1              936      8128187    1984                537   
2             1178    204823716   36068               2669   

   Approved_Conversions  Click-Through Rate (CTR)  Conversion Rate (CR)  \
0                    24                  0.000234              0.513274   
1                   183                  0.000244              0.270665   
2                   872                  0.000176              0.073999   

   Approval Rate (AR)  
0            0.413793  
1            0.340782  
2            0.326714  


Equivalent SQL queries

In [ ]:
-- Overall Funnel Metrics
-- SELECT 
--     SUM(Impressions) AS total_impressions,
--     SUM(Clicks) AS total_clicks,
--     SUM(Total_Conversion) AS total_conversions,
--     SUM(Approved_Conversion) AS total_approved_conversions,
--     ROUND(SUM(Clicks) / NULLIF(SUM(Impressions), 0), 4) AS click_through_rate,
--     ROUND(SUM(Total_Conversion) / NULLIF(SUM(Clicks), 0), 4) AS conversion_rate,
--     ROUND(SUM(Approved_Conversion) / NULLIF(SUM(Total_Conversion), 0), 4) AS approval_rate
-- FROM 
--     conversion_data;

In [ ]:
-- Funnel Metrics by Campaign
-- SELECT 
--     xyz_campaign_id,
--     SUM(Impressions) AS total_impressions,
--     SUM(Clicks) AS total_clicks,
--     SUM(Total_Conversion) AS total_conversions,
--     SUM(Approved_Conversion) AS total_approved_conversions,
--     ROUND(SUM(Clicks) / NULLIF(SUM(Impressions), 0), 4) AS click_through_rate,
--     ROUND(SUM(Total_Conversion) / NULLIF(SUM(Clicks), 0), 4) AS conversion_rate,
--     ROUND(SUM(Approved_Conversion) / NULLIF(SUM(Total_Conversion), 0), 4) AS approval_rate
-- FROM 
--     conversion_data
-- GROUP BY 
--     xyz_campaign_id;

### Customer Acquisition Cost
- On the Overall Data
- Campaign-Level

In [9]:
# Calculate the overall Customer Acquisition Cost (CAC)
total_spent = df['Spent'].sum()
total_approved_conversions = df['Approved_Conversion'].sum()
if total_approved_conversions > 0:
    cac = total_spent / total_approved_conversions
else:
    cac = 0
print(f"The overall Customer Acquisition Cost (CAC) is {cac:.2f}")

The overall Customer Acquisition Cost (CAC) is 54.41


In [11]:
# Calculate Customer Acquisition Cost for each campaign
campaign_cac = df.groupby('xyz_campaign_id').apply(
    lambda x: round(x['Spent'].sum() / x['Approved_Conversion'].sum(), 2) if x['Approved_Conversion'].sum() > 0 else 0
).reset_index(name='CAC')

# Display the CAC for each campaign
print(campaign_cac)

   xyz_campaign_id    CAC
0              916   6.24
1              936  15.81
2             1178  63.83


/var/folders/_r/6_qzzw511b16y7chkmqr51_80000gn/T/ipykernel_1578/579198094.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  campaign_cac = df.groupby('xyz_campaign_id').apply(


Equivalent SQL queries

In [ ]:
-- Overall CAC
-- SELECT 
--     ROUND(SUM(Spent) / NULLIF(SUM(Approved_Conversion), 0), 2) AS overall_cac
-- FROM 
--     conversion_data;

In [ ]:
-- CAC by Campaign
-- SELECT 
--     xyz_campaign_id,
--     ROUND(SUM(Spent) / NULLIF(SUM(Approved_Conversion), 0), 2) AS campaign_cac
-- FROM 
--     conversion_data
-- GROUP BY 
--     xyz_campaign_id;